In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords

from scipy.sparse import hstack, vstack, csr_matrix
from scipy import sparse

from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Dense, Dropout, merge, concatenate
from keras.layers import LSTM, Embedding, GRU
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.layers.wrappers import Bidirectional
from keras import optimizers

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.normalization import BatchNormalization
from keras_tqdm import TQDMNotebookCallback, TQDMCallback


from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.svm import SVC, LinearSVC, NuSVC

from sklearn.metrics import mean_squared_error, accuracy_score, log_loss
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectPercentile, f_classif, mutual_info_classif, chi2, SelectFromModel
from sklearn.feature_selection import VarianceThreshold

from sklearn import preprocessing
from sklearn.decomposition import PCA

import gensim
from gensim.models import Word2Vec

import re
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

from multiprocessing import Pool

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


In [2]:
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt

#font = {'family': 'Verdana', 'weight': 'normal'}
#rc('font', **font)
matplotlib.style.use('ggplot')

%matplotlib inline

pd.set_option('display.max_rows', 15)

# Загрузка данных

### GoogleNews-vectors-negative300

In [3]:
w2v_path = os.path.join(os.sep, os.path.abspath('/home/data/word2vec/english/GoogleNews-vectors-negative300.bin'))
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
w2v.init_sims(replace=True)

### Tokenized Data

In [4]:
def load_tokenized_series(filename):
    try:
        series = pd.read_csv(filename, index_col='id', header=0)
    except:
        series = pd.read_csv(filename, index_col='test_id', header=0)
    series = series['question'].fillna('')
    return series

In [5]:
# load tokenized questions
train_q1_tokenized_objects = load_tokenized_series('train_q1_tokenized.csv')
train_q2_tokenized_objects = load_tokenized_series('train_q2_tokenized.csv')
test_q1_tokenized_objects = load_tokenized_series('test_q1_tokenized.csv')
test_q2_tokenized_objects = load_tokenized_series('test_q2_tokenized.csv')

In [6]:
train_set = pd.read_csv('/home/data/share/quora/train.csv', index_col=0)
Y_train = train_set['is_duplicate']
del train_set

#### stopwords removing

In [7]:
include_stopwords = False

stopwords_set = set([stopword for stopword in stopwords.words("english") if stopword not in w2v.vocab])

def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in stopwords_set])

if not include_stopwords:
    train_q1_tokenized_objects = train_q1_tokenized_objects.apply(remove_stop_words)
    train_q2_tokenized_objects = train_q2_tokenized_objects.apply(remove_stop_words)
    test_q1_tokenized_objects = test_q1_tokenized_objects.apply(remove_stop_words)
    test_q2_tokenized_objects = test_q2_tokenized_objects.apply(remove_stop_words)

In [8]:
stopwords_set

{'a', 'and', 'mightn', 'mustn', 'of', 'to'}

# Множества(train, test, validation)

In [9]:
train_index = train_q1_tokenized_objects.index
train_pairs = pd.DataFrame(index=train_index)
train_pairs['question1'] = train_q1_tokenized_objects
train_pairs['question2'] = train_q2_tokenized_objects
train_pairs = train_pairs.values

test_index = test_q1_tokenized_objects.index
test_pairs = pd.DataFrame(index=test_index)
test_pairs['question1'] = test_q1_tokenized_objects
test_pairs['question2'] = test_q2_tokenized_objects
test_pairs = test_pairs.values

In [10]:
train_tokens, validation_tokens, y_train, y_validation = train_test_split(train_pairs, Y_train.values,
                                                                          test_size=0.1, random_state=42)
test_tokens = test_pairs

# Features

## word2vec squence

In [12]:
def tokens_to_embeddings(text):
    sequence = [w2v.syn0norm[w2v.vocab[token].index] for token in space_tokenizer(text) if word in w2v.vocab]

    if not sequence:
        return np.zeros(300)

    return np.concatenate(sequence)

In [13]:
def space_tokenizer(s):
    return s.split()

In [14]:
all_texts = np.concatenate([train_pairs[:, 0], train_pairs[:, 1], test_pairs[:, 0], test_pairs[:, 1]])
tokenizer = Tokenizer(filters='', lower=False, split=' ')
tokenizer.fit_on_texts(all_texts)

In [15]:
vocab = tokenizer.word_index
vocab_len = len(vocab)
print('Found {} unique tokens'.format(vocab_len))

Found 128778 unique tokens


In [16]:
train_sequences = train_tokens.copy()
train_sequences[:, 0] = tokenizer.texts_to_sequences(train_sequences[:, 0])
train_sequences[:, 1] = tokenizer.texts_to_sequences(train_sequences[:, 1])

validation_sequences = validation_tokens.copy()
validation_sequences[:, 0] = tokenizer.texts_to_sequences(validation_sequences[:, 0])
validation_sequences[:, 1] = tokenizer.texts_to_sequences(validation_sequences[:, 1])

test_sequences = test_tokens.copy()
test_sequences[:, 0] = tokenizer.texts_to_sequences(test_sequences[:, 0])
test_sequences[:, 1] = tokenizer.texts_to_sequences(test_sequences[:, 1])

In [17]:
embedding_dim = 300
embedding_weights = np.zeros((vocab_len+1, embedding_dim))
for word, index in vocab.items():
    if word in w2v.vocab:
        embedding_weights[index, :] = w2v.syn0norm[w2v.vocab[word].index] 
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_weights, axis=1) == 0))

Null word embeddings: 69934


In [18]:
maxlen = 25

train_sequences_q1 = pad_sequences(train_sequences[:, 0], maxlen=maxlen)
train_sequences_q2 = pad_sequences(train_sequences[:, 1], maxlen=maxlen)
train_sequences = np.array([pair for pair in list(zip(train_sequences_q1, train_sequences_q2))])
del train_sequences_q1, train_sequences_q2

validation_sequences_q1 = pad_sequences(validation_sequences[:, 0], maxlen=maxlen)
validation_sequences_q2 = pad_sequences(validation_sequences[:, 1], maxlen=maxlen)
validation_sequences = np.array([pair for pair in list(zip(validation_sequences_q1, validation_sequences_q2))])
del validation_sequences_q1, validation_sequences_q2

test_sequences_q1 = pad_sequences(test_sequences[:, 0], maxlen=maxlen)
test_sequences_q2 = pad_sequences(test_sequences[:, 1], maxlen=maxlen)
test_sequences = np.array([pair for pair in list(zip(test_sequences_q1, test_sequences_q2))])
del test_sequences_q1, test_sequences_q2

In [19]:
train_sequences.shape, validation_sequences.shape, test_sequences.shape

((363861, 2, 25), (40429, 2, 25), (2345796, 2, 25))

In [20]:
x_train = train_sequences
x_validation = validation_sequences
x_test = test_sequences

# Models

# NN

In [21]:
max_features, embedding_dim, maxlen = vocab_len+1, 300, maxlen

## With Embedding

### batch_generator

In [22]:
def batch_generator(Q1, Q2, Y, batch_size, shuffle=False):
    n = Y.shape[0]
    number_of_batches, counter, shuffle_index = n//batch_size, 0, np.arange(n)
    
    if shuffle:
        np.random.shuffle(shuffle_index)

    Q1, Q2, Y =  Q1[shuffle_index], Q2[shuffle_index], Y[shuffle_index]
    
    while True:
        for counter in range(number_of_batches+1):
            index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
            X, Y_batch = [Q1[index_batch], Q2[index_batch]], Y[index_batch]
            yield X, Y_batch
            
        if shuffle:
            np.random.shuffle(shuffle_index)

### model architecture

In [23]:
def get_representation(input_shape):
    representation = Sequential()
    representation.add(Embedding(max_features, embedding_dim, input_length=input_shape, 
                                 weights=[embedding_weights], trainable=False))
    representation.add(GRU(embedding_dim, dropout=0.25, recurrent_dropout=0.25))
    
    return representation

def build_model(question_shape):
    first_question = Input(shape=(question_shape, ))
    second_question = Input(shape=(question_shape, ))
    
    representation = get_representation(question_shape)
    
    first_representation = representation(first_question)
    second_representation = representation(second_question)
    
    merged_vectors = concatenate([first_representation, second_representation])
    features = Dropout(0.3)(merged_vectors)
    features = BatchNormalization()(features)

    features = Dense(100, activation='relu')(features)
    features = Dropout(0.3)(features)
    features = BatchNormalization()(features)
    
    output_proba = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(features)
    
    model = Model(
        inputs=[first_question, second_question],
        outputs=output_proba
    )
    
    return model
    
def get_nn_model(*, path_to_hdf5=False, input_shape=None):
    model = build_model(input_shape) 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # load pretrained weights
    if path_to_hdf5:
        model.load_weights(path_to_hdf5)

    print(model.summary())
    return model

filepath="weights_{epoch:02d}_{val_loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
callbacks_list = [checkpoint, early_stopping, TQDMNotebookCallback()]

### training

In [24]:
batch_size = 1024
steps_per_epoch = y_train.shape[0]//batch_size
validation_steps = y_validation.shape[0]//batch_size
input_shape = maxlen

train_generator = batch_generator(x_train[:, 0], x_train[:, 1], y_train, batch_size, shuffle=False)
validation_generator = batch_generator(x_validation[:, 0], x_validation[:, 1], y_validation, batch_size, shuffle=False)

model = get_nn_model(input_shape=input_shape)
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                              epochs=200, verbose=2, callbacks=callbacks_list,
                              max_q_size=2)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 25)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 25)            0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 300)           39174600                                     
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 600)           0                                            
___________________________________________________________________________________________

The installed widget Javascript is the wrong version.
The installed widget Javascript is the wrong version.


Epoch 1/200
Epoch 00000: val_loss improved from inf to 0.59114, saving model to weights_00_0.591.hdf5
54s - loss: 0.5979 - acc: 0.6831 - val_loss: 0.5911 - val_acc: 0.6661
Epoch 2/200


The installed widget Javascript is the wrong version.


Epoch 00001: val_loss improved from 0.59114 to 0.51422, saving model to weights_01_0.514.hdf5
54s - loss: 0.5369 - acc: 0.7263 - val_loss: 0.5142 - val_acc: 0.7487
Epoch 3/200


...

Epoch 00069: val_loss did not improve
54s - loss: 0.2969 - acc: 0.8669 - val_loss: 0.3404 - val_acc: 0.8442
Epoch 71/200


The installed widget Javascript is the wrong version.


Epoch 00070: val_loss did not improve
54s - loss: 0.2966 - acc: 0.8675 - val_loss: 0.3450 - val_acc: 0.8421
Epoch 72/200


The installed widget Javascript is the wrong version.


Epoch 00071: val_loss did not improve
54s - loss: 0.2963 - acc: 0.8679 - val_loss: 0.3429 - val_acc: 0.8437

